In [12]:
from functions import download_youtube
video_id = 'ek05M8eCk7M'
#video_id = 'I-cxigjLG0I'
youtube_url = f"https://www.youtube.com/watch?v={video_id}"
job_id = 'Hanni'
file_path, file_name =  download_youtube(youtube_url, job_id=job_id)
file_name

[youtube] Extracting URL: https://www.youtube.com/watch?v=ek05M8eCk7M
[youtube] ek05M8eCk7M: Downloading webpage
[youtube] ek05M8eCk7M: Downloading ios player API JSON
[youtube] ek05M8eCk7M: Downloading player e38bb6de


         n = jrX2HCYBWf_MoznJ ; player = https://www.youtube.com/s/player/e38bb6de/player_ias.vflset/en_US/base.js
         n = fMwAfFGhnG5jzmuK ; player = https://www.youtube.com/s/player/e38bb6de/player_ias.vflset/en_US/base.js


[youtube] ek05M8eCk7M: Downloading m3u8 information
[info] ek05M8eCk7M: Downloading 1 format(s): 625+140
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 50
[download] Destination: Hanni_ek05M8eCk7M.f625.mp4
[download] 100% of  551.14MiB in 00:00:50 at 11.00MiB/s                
[download] Destination: Hanni_ek05M8eCk7M.f140.m4a
[download] 100% of    4.29MiB in 00:00:00 at 12.40MiB/s  
[Merger] Merging formats into "Hanni_ek05M8eCk7M.mp4"
Deleting original file Hanni_ek05M8eCk7M.f625.mp4 (pass -k to keep)
Deleting original file Hanni_ek05M8eCk7M.f140.m4a (pass -k to keep)


'Hanni_ek05M8eCk7M.mp4'

In [13]:
from functions import slice_video
slice_video(file_name, [(0, 240, 'hanni.mp4')], height=540, width=960)

2024-09-13 14:22:39,872 - INFO - Successfully created segment: hanni.mp4


[{'output_name': 'hanni.mp4',
  'start': 0,
  'duration': 240,
  'success': True,
  'error': None}]

In [15]:
base_image = "../face_recognition/test_hanni2.jpg"
#target_video = "hanni.mp4"
target_video = "삼성SDS_FabriX.mp4"

import cv2
import face_recognition

# 특정 인물의 이미지를 로드하여 인코딩
known_image = face_recognition.load_image_file(base_image)
known_face_encoding = face_recognition.face_encodings(known_image)[0]

# 비디오 파일 로드
video_capture = cv2.VideoCapture(target_video)

# 변수 초기화
trackers = []
face_names = []
face_distances = []
frame_skip = 12
frame_count = 0

while True:
    # 비디오에서 한 프레임씩 읽기
    ret, frame = video_capture.read()

    if not ret:
        break

    frame_count += 1

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    if frame_count % frame_skip == 0:
        # 추적기와 얼굴 정보 초기화
        trackers = []
        face_names = []
        face_distances = []

        # 얼굴 위치와 인코딩 탐지
        face_locations = face_recognition.face_locations(rgb_frame)
        face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

        for face_location, face_encoding in zip(face_locations, face_encodings):
            top, right, bottom, left = face_location

            # 얼굴 거리 계산
            distance = face_recognition.face_distance([known_face_encoding], face_encoding)[0]

            # 임계값 설정 (예: 0.6)
            tolerance = 0.46

            if distance < tolerance:
                name = "Specific Person"
            else:
                name = "Unknown"

            # 추적기 초기화
            tracker = cv2.legacy.TrackerKCF_create()
            bbox = (left, top, right - left, bottom - top)
            tracker.init(frame, bbox)
            trackers.append(tracker)
            face_names.append(name)
            face_distances.append(distance)
    else:
        # 모든 추적기 업데이트
        new_trackers = []
        new_face_names = []
        new_face_distances = []

        for tracker, name, distance in zip(trackers, face_names, face_distances):
            success, bbox = tracker.update(frame)
            if success:
                new_trackers.append(tracker)
                new_face_names.append(name)
                new_face_distances.append(distance)
                left, top, width, height = [int(v) for v in bbox]
                right = left + width
                bottom = top + height

                # 사각형과 텍스트 표시
                if name == "Specific Person":
                    # 특정 인물에 대해 빨간색 사각형과 이름 및 거리 표시
                    cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
                    # 이름 표시
                    cv2.putText(frame, name, (left, bottom + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
                    # 거리 표시 (이름 아래)
                    cv2.putText(frame, f"Distance: {distance:.2f}", (left, bottom + 45), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
                else:
                    # 다른 인물에 대해 초록색 사각형과 이름만 표시
                    cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
                    cv2.putText(frame, name, (left, bottom + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
                    # 거리 표시 (이름 아래)
                    cv2.putText(frame, f"Distance: {distance:.2f}", (left, bottom + 45), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        # 추적기와 얼굴 정보 업데이트
        trackers = new_trackers
        face_names = new_face_names
        face_distances = new_face_distances

    # 비디오 출력
    cv2.imshow('Video', frame)

    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 모든 작업 완료 후 클린업
video_capture.release()
cv2.destroyAllWindows()


In [20]:
source_file = "../face_recognition/faces/newjeans01_g.jpg"
known_file = "../face_recognition/test_hanni2.jpg"
target_file = "../face_recognition/test_yun.jpg"

import face_recognition
import cv2
import numpy as np

# 소스 이미지 로드 (얼굴을 교체할 이미지)
source_image = face_recognition.load_image_file(source_file)
source_image_rgb = source_image[:, :, ::-1]

# 대상 얼굴 이미지 로드 (교체할 얼굴)
target_face_image = face_recognition.load_image_file(target_file)
target_face_locations = face_recognition.face_locations(target_face_image)
if len(target_face_locations) == 0:
    print("대상 얼굴 이미지에서 얼굴을 찾을 수 없습니다.")
    exit()

# 대상 얼굴의 첫 번째 얼굴 선택
target_face_location = target_face_locations[0]
top_t, right_t, bottom_t, left_t = target_face_location
target_face = target_face_image[top_t:bottom_t, left_t:right_t]

# 교체할 얼굴 로드 (소스 이미지에서 교체하려는 특정 얼굴)
known_face_image = face_recognition.load_image_file(known_file)
known_face_encodings = face_recognition.face_encodings(known_face_image)
if len(known_face_encodings) == 0:
    print("교체할 얼굴 이미지에서 얼굴을 찾을 수 없습니다.")
    exit()
known_face_encoding = known_face_encodings[0]

# 소스 이미지에서 모든 얼굴과 인코딩 찾기
source_face_locations = face_recognition.face_locations(source_image)
source_face_encodings = face_recognition.face_encodings(source_image, source_face_locations)

# 소스 이미지의 각 얼굴에 대해 반복
for (face_location, face_encoding) in zip(source_face_locations, source_face_encodings):
    # 이 얼굴이 교체하려는 얼굴인지 확인
    matches = face_recognition.compare_faces([known_face_encoding], face_encoding, tolerance=0.4)

    if matches[0]:
        # 교체할 얼굴을 찾았습니다.
        top, right, bottom, left = face_location
        # 대상 얼굴을 교체할 얼굴의 크기에 맞게 조정
        face_width = right - left
        face_height = bottom - top
        target_face_resized = cv2.resize(target_face, (face_width, face_height))

        # 소스 이미지에서 얼굴 교체
        source_image[top:bottom, left:right] = target_face_resized

# 결과 저장
cv2.imwrite("result.jpg", cv2.cvtColor(source_image, cv2.COLOR_RGB2BGR))


True